In [12]:
%%capture

# !pip install snowflake-connector-python
!pip install "snowflake-connector-python[secure-local-storage,pandas]"

In [2]:
%%capture

!pip install snowflake-sqlalchemy

In [3]:
# %%capture

# !pip install xgboost

In [1]:
import os
import pickle
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn import metrics

import sqlalchemy
import snowflake.connector
from sqlalchemy import create_engine
from snowflake.sqlalchemy import *

import xgboost
from datetime import datetime, timedelta
import time  
import pytz    
tz_NY = pytz.timezone('Asia/Kolkata')

import snowflake_creds
import LOS_Preprocessing
from LOS_Preprocessing import preprocess_data

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Creating the connection engine (way 1)
engine = create_engine(URL(
        account="tr09543.ap-south-1",
        user= snowflake_creds.USER_NAME,
        password= snowflake_creds.PASSWORD,
        role="ACCOUNTADMIN",
        warehouse="COMPUTE_WH",
        database="HEALTHDB",
        schema="HEALTHSCHEMA"
    ))

In [3]:
query = """

WITH BASE AS (

    SELECT CASE_ID,
           COALESCE(HOSPITAL_CODE,0) AS HOSPITAL_CODE,
           COALESCE(HOSPITAL_TYPE_CODE,'None') AS HOSPITAL_TYPE_CODE,
           COALESCE(CITY_CODE_HOSPITAL,0) AS CITY_CODE_HOSPITAL,
           COALESCE(HOSPITAL_REGION_CODE,'None') AS HOSPITAL_REGION_CODE,
           COALESCE(AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL,0) AS AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL,
           COALESCE(DEPARTMENT,'None') AS DEPARTMENT,
           COALESCE(WARD_TYPE,'None') AS WARD_TYPE,
           COALESCE(WARD_FACILITY_CODE,'None') AS WARD_FACILITY_CODE,
           COALESCE(BED_GRADE,0) AS BED_GRADE,
           PATIENTID,
           COALESCE(CITY_CODE_PATIENT,0) AS CITY_CODE_PATIENT,
           COALESCE(TYPE_OF_ADMISSION,'None') AS TYPE_OF_ADMISSION,
           COALESCE(SEVERITY_OF_ILLNESS,'Minor') AS SEVERITY_OF_ILLNESS,
           COALESCE(VISITORS_WITH_PATIENT,0) AS VISITORS_WITH_PATIENT,
           COALESCE(AGE,'None') AS AGE,
           COALESCE(ADMISSION_DEPOSIT,0) AS ADMISSION_DEPOSIT,
           ADMISSION_DATE,
           DISCHARGE_DATE

    FROM HEALTHDB.HEALTHSCHEMA.SIMULATION_DATA

),
    
BASE_WITH_FEATURES AS (

    SELECT *,
            MONTHNAME(ADMISSION_DATE) AS ADMISSION_MONTH,
            DAYNAME(ADMISSION_DATE) AS ADMISSION_DAY,
            CONCAT(TYPE_OF_ADMISSION,'-',SEVERITY_OF_ILLNESS) AS ADMISSION_ILLNESS,
            CONCAT(SEVERITY_OF_ILLNESS,'-',BED_GRADE) AS ILLNESS_BEDGRADE,
            CONCAT(DEPARTMENT,'-',SEVERITY_OF_ILLNESS) AS DEPARTMENT_ILLNESS,
            DATEDIFF(day,ADMISSION_DATE,DISCHARGE_DATE) AS LOS
    FROM BASE 

)    

SELECT * FROM BASE_WITH_FEATURES WHERE ADMISSION_DATE = CURRENT_DATE-45
"""

In [13]:
def check_n_create_model_features(df,feat_list):
    test = pd.DataFrame()
    for col in feat_list:
        if col in df.columns.tolist():
            test[col] = df[col]
        else:
            test[col] = 0
    
    return test

In [14]:
def insert_predictions_to_snowflake_table(data):
    import pandas
    import snowflake.connector
    from snowflake.connector.pandas_tools import pd_writer, write_pandas

    engine = create_engine(URL(
            account="tr09543.ap-south-1",
            user= snowflake_creds.USER_NAME,
            password= snowflake_creds.PASSWORD,
            role="ACCOUNTADMIN",
            warehouse="COMPUTE_WH",
            database="HEALTHDB",
            schema="HEALTHSCHEMA"
        ))  
    
    # Creating the logging table if not exists already
    table = 'TEMP_LOS_PREDICTION_MODEL_LOGGING_TABLE_HARI'
    
    # Inserting the data to snowflake logging table
    data.to_sql(table, engine, index=False, if_exists='append', method=pd_writer)
    return 'Success'

In [18]:
with engine.connect() as conn:
    
    # Loading the scoring data
    score_data = pd.DataFrame(pd.read_sql(query,conn))
    score_data.columns = [col.upper() for col in score_data.columns.tolist()]
    
    # Applying the preprocessing steps
    score_data_processed = LOS_Preprocessing.preprocess_data(score_data)
    
    # Applying feature selection
    final_feats = pd.read_pickle('MODEL_FEATS.pkl')
    score_data_final = check_n_create_model_features(score_data_processed,final_feats)
    
    # Getting the predictions
    model = xgboost.XGBRegressor()
    model.load_model('MODEL_XGB.model')
    score_data_final['PREDICTED_LOS'] = np.ceil(model.predict(score_data_final.drop('LOS',axis=1)))
    
    # Writing the dataframe to snowflake as a table
    score_data_final = score_data_final.reset_index()
    score_data_table = pd.merge(score_data,score_data_final,on='CASE_ID',how='left')
    status = insert_predictions_to_snowflake_table(score_data_table)

In [19]:
print(score_data.shape)
score_data.head()

(2362, 25)


,CASE_ID,HOSPITAL_CODE,HOSPITAL_TYPE_CODE,CITY_CODE_HOSPITAL,HOSPITAL_REGION_CODE,AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL,DEPARTMENT,WARD_TYPE,WARD_FACILITY_CODE,BED_GRADE,PATIENTID,CITY_CODE_PATIENT,TYPE_OF_ADMISSION,SEVERITY_OF_ILLNESS,VISITORS_WITH_PATIENT,AGE,ADMISSION_DEPOSIT,ADMISSION_DATE,DISCHARGE_DATE,ADMISSION_MONTH,ADMISSION_DAY,ADMISSION_ILLNESS,ILLNESS_BEDGRADE,DEPARTMENT_ILLNESS,LOS
0,1,19,a,7,Y,4,gynecology,Q,C,3,48585,8,Emergency,Moderate,2,61-70,3827,2022-12-01,2022-12-31,Dec,Thu,Emergency-Moderate,Moderate-3,gynecology-Moderate,30
1,2,26,b,2,Y,5,gynecology,R,D,2,48585,8,Urgent,Moderate,4,61-70,4677,2022-12-01,2023-01-10,Dec,Thu,Urgent-Moderate,Moderate-2,gynecology-Moderate,40
2,3,19,a,7,Y,2,gynecology,S,C,2,110115,8,Emergency,Moderate,4,71-80,3924,2022-12-01,2023-01-30,Dec,Thu,Emergency-Moderate,Moderate-2,gynecology-Moderate,60
3,4,18,d,13,Y,2,radiotherapy,R,B,2,127970,3,Emergency,Moderate,2,31-40,5124,2022-12-01,2022-12-11,Dec,Thu,Emergency-Moderate,Moderate-2,radiotherapy-Moderate,10
4,5,23,a,6,X,5,gynecology,Q,F,4,130244,2,Trauma,Moderate,4,21-30,4527,2022-12-01,2022-12-21,Dec,Thu,Trauma-Moderate,Moderate-4,gynecology-Moderate,20


In [20]:
print(score_data_processed.shape)
score_data_processed.head()

(2362, 129)


,AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL,VISITORS_WITH_PATIENT,ADMISSION_DEPOSIT,LOS,HOSPITAL_TYPE_CODE_a,HOSPITAL_TYPE_CODE_b,HOSPITAL_TYPE_CODE_c,HOSPITAL_TYPE_CODE_d,HOSPITAL_TYPE_CODE_e,HOSPITAL_TYPE_CODE_f,HOSPITAL_TYPE_CODE_g,CITY_CODE_HOSPITAL_1,CITY_CODE_HOSPITAL_2,CITY_CODE_HOSPITAL_3,CITY_CODE_HOSPITAL_4,CITY_CODE_HOSPITAL_5,CITY_CODE_HOSPITAL_6,CITY_CODE_HOSPITAL_7,CITY_CODE_HOSPITAL_9,CITY_CODE_HOSPITAL_10,CITY_CODE_HOSPITAL_11,CITY_CODE_HOSPITAL_13,HOSPITAL_REGION_CODE_X,HOSPITAL_REGION_CODE_Y,HOSPITAL_REGION_CODE_Z,DEPARTMENT_TB & Chest disease,DEPARTMENT_anesthesia,DEPARTMENT_gynecology,DEPARTMENT_radiotherapy,DEPARTMENT_surgery,WARD_TYPE_P,WARD_TYPE_Q,WARD_TYPE_R,WARD_TYPE_S,WARD_TYPE_T,WARD_FACILITY_CODE_A,WARD_FACILITY_CODE_B,WARD_FACILITY_CODE_C,WARD_FACILITY_CODE_D,WARD_FACILITY_CODE_E,WARD_FACILITY_CODE_F,BED_GRADE_1,BED_GRADE_2,BED_GRADE_3,BED_GRADE_4,CITY_CODE_PATIENT_0,CITY_CODE_PATIENT_1,CITY_CODE_PATIENT_2,CITY_CODE_PATIENT_3,CITY_CODE_PATIENT_4,CITY_CODE_PATIENT_5,CITY_CODE_PATIENT_6,CITY_CODE_PATIENT_7,CITY_CODE_PATIENT_8,CITY_CODE_PATIENT_9,CITY_CODE_PATIENT_10,CITY_CODE_PATIENT_11,CITY_CODE_PATIENT_12,CITY_CODE_PATIENT_13,CITY_CODE_PATIENT_14,CITY_CODE_PATIENT_15,CITY_CODE_PATIENT_16,CITY_CODE_PATIENT_18,CITY_CODE_PATIENT_19,CITY_CODE_PATIENT_20,CITY_CODE_PATIENT_21,CITY_CODE_PATIENT_22,CITY_CODE_PATIENT_23,CITY_CODE_PATIENT_24,CITY_CODE_PATIENT_25,CITY_CODE_PATIENT_26,CITY_CODE_PATIENT_27,CITY_CODE_PATIENT_28,CITY_CODE_PATIENT_29,CITY_CODE_PATIENT_30,CITY_CODE_PATIENT_33,TYPE_OF_ADMISSION_Emergency,TYPE_OF_ADMISSION_Trauma,TYPE_OF_ADMISSION_Urgent,SEVERITY_OF_ILLNESS_Extreme,SEVERITY_OF_ILLNESS_Minor,SEVERITY_OF_ILLNESS_Moderate,AGE_0-10,AGE_20-Nov,AGE_21-30,AGE_31-40,AGE_41-50,AGE_51-60,AGE_61-70,AGE_71-80,AGE_81-90,AGE_91-100,ADMISSION_MONTH_Dec,ADMISSION_DAY_Thu,ADMISSION_ILLNESS_Emergency-Extreme,ADMISSION_ILLNESS_Emergency-Minor,ADMISSION_ILLNESS_Emergency-Moderate,ADMISSION_ILLNESS_Trauma-Extreme,ADMISSION_ILLNESS_Trauma-Minor,ADMISSION_ILLNESS_Trauma-Moderate,ADMISSION_ILLNESS_Urgent-Extreme,ADMISSION_ILLNESS_Urgent-Minor,ADMISSION_ILLNESS_Urgent-Moderate,ILLNESS_BEDGRADE_Extreme-1,ILLNESS_BEDGRADE_Extreme-2,ILLNESS_BEDGRADE_Extreme-3,ILLNESS_BEDGRADE_Extreme-4,ILLNESS_BEDGRADE_Minor-1,ILLNESS_BEDGRADE_Minor-2,ILLNESS_BEDGRADE_Minor-3,ILLNESS_BEDGRADE_Minor-4,ILLNESS_BEDGRADE_Moderate-1,ILLNESS_BEDGRADE_Moderate-2,ILLNESS_BEDGRADE_Moderate-3,ILLNESS_BEDGRADE_Moderate-4,DEPARTMENT_ILLNESS_TB & Chest disease-Extreme,DEPARTMENT_ILLNESS_TB & Chest disease-Minor,DEPARTMENT_ILLNESS_TB & Chest disease-Moderate,DEPARTMENT_ILLNESS_anesthesia-Extreme,DEPARTMENT_ILLNESS_anesthesia-Minor,DEPARTMENT_ILLNESS_anesthesia-Moderate,DEPARTMENT_ILLNESS_gynecology-Extreme,DEPARTMENT_ILLNESS_gynecology-Minor,DEPARTMENT_ILLNESS_gynecology-Moderate,DEPARTMENT_ILLNESS_radiotherapy-Extreme,DEPARTMENT_ILLNESS_radiotherapy-Minor,DEPARTMENT_ILLNESS_radiotherapy-Moderate,DEPARTMENT_ILLNESS_surgery-Extreme,DEPARTMENT_ILLNESS_surgery-Moderate
CASE_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,4,2,3827,30,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,5,4,4677,40,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,2,4,3924,60,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,2,2,5124,10,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,

In [21]:
print(score_data_final.shape)
score_data_final.head()

(2362, 30)


,CASE_ID,TYPE_OF_ADMISSION_Trauma,ADMISSION_DAY_Thu,ADMISSION_DAY_Sat,AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL,ADMISSION_DAY_Tue,SEVERITY_OF_ILLNESS_Minor,BED_GRADE_2,ADMISSION_MONTH_Oct,VISITORS_WITH_PATIENT,CITY_CODE_HOSPITAL_7,DEPARTMENT_ILLNESS_gynecology-Extreme,AGE_31-40,AGE_41-50,ADMISSION_MONTH_Nov,TYPE_OF_ADMISSION_Emergency,ADMISSION_DEPOSIT,ADMISSION_DAY_Wed,ILLNESS_BEDGRADE_Extreme-1,AGE_71-80,WARD_TYPE_S,AGE_51-60,WARD_TYPE_Q,WARD_TYPE_P,ADMISSION_DAY_Fri,ADMISSION_DAY_Sun,ADMISSION_DAY_Mon,CITY_CODE_PATIENT_8,LOS,PREDICTED_LOS
0,1,0,1,0,4,0,0,0,0,2,1,0,0,0,0,1,3827,0,0,0,0,0,1,0,0,0,0,1,30,22.0
1,2,0,1,0,5,0,0,1,0,4,0,0,0,0,0,0,4677,0,0,0,0,0,0,0,0,0,0,1,40,55.0
2,3,0,1,0,2,0,0,1,0,4,1,0,0,0,0,1,3924,0,0,1,1,0,0,0,0,0,0,1,60,45.0
3,4,0,1,0,2,0,0,1,0,2,0,0,1,0,0,1,5124,0,0,0,0,0,0,0,0,0,0,0,10,28.0
4,5,1,1,0,5,0,0,0,0,4,0,0,0,0,0,0,4527,0,0,0,0,0,1,0,0,0,0,0,20,30.0


In [22]:
print(score_data_table.shape)
score_data_table.head()

(2362, 54)


,CASE_ID,HOSPITAL_CODE,HOSPITAL_TYPE_CODE,CITY_CODE_HOSPITAL,HOSPITAL_REGION_CODE,AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL_x,DEPARTMENT,WARD_TYPE,WARD_FACILITY_CODE,BED_GRADE,PATIENTID,CITY_CODE_PATIENT,TYPE_OF_ADMISSION,SEVERITY_OF_ILLNESS,VISITORS_WITH_PATIENT_x,AGE,ADMISSION_DEPOSIT_x,ADMISSION_DATE,DISCHARGE_DATE,ADMISSION_MONTH,ADMISSION_DAY,ADMISSION_ILLNESS,ILLNESS_BEDGRADE,DEPARTMENT_ILLNESS,LOS_x,TYPE_OF_ADMISSION_Trauma,ADMISSION_DAY_Thu,ADMISSION_DAY_Sat,AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL_y,ADMISSION_DAY_Tue,SEVERITY_OF_ILLNESS_Minor,BED_GRADE_2,ADMISSION_MONTH_Oct,VISITORS_WITH_PATIENT_y,CITY_CODE_HOSPITAL_7,DEPARTMENT_ILLNESS_gynecology-Extreme,AGE_31-40,AGE_41-50,ADMISSION_MONTH_Nov,TYPE_OF_ADMISSION_Emergency,ADMISSION_DEPOSIT_y,ADMISSION_DAY_Wed,ILLNESS_BEDGRADE_Extreme-1,AGE_71-80,WARD_TYPE_S,AGE_51-60,WARD_TYPE_Q,WARD_TYPE_P,ADMISSION_DAY_Fri,ADMISSION_DAY_Sun,ADMISSION_DAY_Mon,CITY_CODE_PATIENT_8,LOS_y,PREDICTED_LOS
0,1,19,a,7,Y,4,gynecology,Q,C,3,48585,8,Emergency,Moderate,2,61-70,3827,2022-12-01,2022-12-31,Dec,Thu,Emergency-Moderate,Moderate-3,gynecology-Moderate,30,0,1,0,4,0,0,0,0,2,1,0,0,0,0,1,3827,0,0,0,0,0,1,0,0,0,0,1,30,22.0
1,2,26,b,2,Y,5,gynecology,R,D,2,48585,8,Urgent,Moderate,4,61-70,4677,2022-12-01,2023-01-10,Dec,Thu,Urgent-Moderate,Moderate-2,gynecology-Moderate,40,0,1,0,5,0,0,1,0,4,0,0,0,0,0,0,4677,0,0,0,0,0,0,0,0,0,0,1,40,55.0
2,3,19,a,7,Y,2,gynecology,S,C,2,110115,8,Emergency,Moderate,4,71-80,3924,2022-12-01,2023-01-30,Dec,Thu,Emergency-Moderate,Moderate-2,gynecology-Moderate,60,0,1,0,2,0,0,1,0,4,1,0,0,0,0,1,3924,0,0,1,1,0,0,0,0,0,0,1,60,45.0
3,4,18,d,13,Y,2,radiotherapy,R,B,2,127970,3,Emergency,Moderate,2,31-40,5124,2022-12-01,2022-12-11,Dec,Thu,Emergency-Moderate,Moderate-2,radiotherapy-Moderate,10,0,1,0,2,0,0,1,0,2,0,0,1,0,0,1,5124,0,0,0,0,0,0,0,0,0,0,0,10,28.0
4,5,23,a,6,X,5,gynecology,Q,F,4,130244,2,Trauma,Moderate,4,21-30,4527,2022-12-01,2022-12-21,Dec,Thu,Trauma-Moderate,Moderate-4,gynecology-Moderate,20,1,1,0,5,0,0,0,0,4,0,0,0,0,0,0,4527,0,0,0,0,0,1,0,0,0,0,0,20,30.0


# Scoring Function for Deployment:

In [23]:
query = """

WITH BASE AS (

    SELECT CASE_ID,
           COALESCE(HOSPITAL_CODE,0) AS HOSPITAL_CODE,
           COALESCE(HOSPITAL_TYPE_CODE,'None') AS HOSPITAL_TYPE_CODE,
           COALESCE(CITY_CODE_HOSPITAL,0) AS CITY_CODE_HOSPITAL,
           COALESCE(HOSPITAL_REGION_CODE,'None') AS HOSPITAL_REGION_CODE,
           COALESCE(AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL,0) AS AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL,
           COALESCE(DEPARTMENT,'None') AS DEPARTMENT,
           COALESCE(WARD_TYPE,'None') AS WARD_TYPE,
           COALESCE(WARD_FACILITY_CODE,'None') AS WARD_FACILITY_CODE,
           COALESCE(BED_GRADE,0) AS BED_GRADE,
           PATIENTID,
           COALESCE(CITY_CODE_PATIENT,0) AS CITY_CODE_PATIENT,
           COALESCE(TYPE_OF_ADMISSION,'None') AS TYPE_OF_ADMISSION,
           COALESCE(SEVERITY_OF_ILLNESS,'Minor') AS SEVERITY_OF_ILLNESS,
           COALESCE(VISITORS_WITH_PATIENT,0) AS VISITORS_WITH_PATIENT,
           COALESCE(AGE,'None') AS AGE,
           COALESCE(ADMISSION_DEPOSIT,0) AS ADMISSION_DEPOSIT,
           ADMISSION_DATE,
           DISCHARGE_DATE

    FROM HEALTHDB.HEALTHSCHEMA.SIMULATION_DATA

),
    
BASE_WITH_FEATURES AS (

    SELECT *,
            MONTHNAME(ADMISSION_DATE) AS ADMISSION_MONTH,
            DAYNAME(ADMISSION_DATE) AS ADMISSION_DAY,
            CONCAT(TYPE_OF_ADMISSION,'-',SEVERITY_OF_ILLNESS) AS ADMISSION_ILLNESS,
            CONCAT(SEVERITY_OF_ILLNESS,'-',BED_GRADE) AS ILLNESS_BEDGRADE,
            CONCAT(DEPARTMENT,'-',SEVERITY_OF_ILLNESS) AS DEPARTMENT_ILLNESS,
            DATEDIFF(day,ADMISSION_DATE,DISCHARGE_DATE) AS LOS
    FROM BASE 

)    

SELECT * FROM BASE_WITH_FEATURES WHERE ADMISSION_DATE = CURRENT_DATE-45
"""

In [24]:
def check_n_create_model_features(df,feat_list):
    test = pd.DataFrame()
    for col in feat_list:
        if col in df.columns.tolist():
            test[col] = df[col]
        else:
            test[col] = 0
    
    return test

In [25]:
def insert_predictions_to_snowflake_table(data):
    import pandas
    import snowflake.connector
    from snowflake.connector.pandas_tools import pd_writer, write_pandas

    engine = create_engine(URL(
            account="tr09543.ap-south-1",
            user= snowflake_creds.USER_NAME,
            password= snowflake_creds.PASSWORD,
            role="ACCOUNTADMIN",
            warehouse="COMPUTE_WH",
            database="HEALTHDB",
            schema="HEALTHSCHEMA"
        ))  
    
    # Creating the logging table if not exists already
    table = 'TEMP_LOS_PREDICTION_MODEL_LOGGING_TABLE_HARI'
    
    # Inserting the data to snowflake logging table
    data.to_sql(table, engine, index=False, if_exists='append', method=pd_writer)
    return 'Success'

In [26]:
def send_status_mail(mail_string):
    import mail_creds
    import smtplib
    from email.mime.multipart import MIMEMultipart
    from email.mime.text import MIMEText
    from email.mime.multipart import MIMEMultipart
    from email.mime.base import MIMEBase
    from email.mime.text import MIMEText
    from email.utils import formatdate
    from email import encoders
    
    subject = 'Patient LOS Prediction - STATUS MAIL'
    mail_content = mail_string

    username= mail_creds.MAIL_ID
    password= mail_creds.APP_PASSWORD
    send_from = mail_creds.MAIL_ID
    send_to = mail_creds.MAIL_ID
    Cc = ''
    msg = MIMEMultipart()
    msg['From'] = send_from
    msg['To'] = send_to
    msg['Cc'] = Cc
    msg['Date'] = formatdate(localtime = True)
    msg['Subject'] = subject
    msg.attach(MIMEText(mail_content, 'plain'))
    smtp = smtplib.SMTP('smtp.gmail.com',587)
    smtp.ehlo()
    smtp.starttls()
    smtp.login(username,password)
    smtp.sendmail(send_from, send_to.split(',') + msg['Cc'].split(','), msg.as_string())
    smtp.quit()    

In [29]:
def LOS_MODEL_DEPLOYMENT():
    try:
        import pickle
        import pandas as pd
        import numpy as np
        import sqlalchemy
        import snowflake.connector
        from sqlalchemy import create_engine

        import xgboost as xgb
        from datetime import datetime, timedelta
        import time  
        import pytz    
        tz_NY = pytz.timezone('Asia/Kolkata')

        import warnings
        warnings.filterwarnings('ignore')   
        
        import snowflake_creds
        import LOS_Preprocessing
        from LOS_Preprocessing import preprocess_data
        
        # Creating the connection engine (way 1)
        engine = create_engine(URL(
                account="tr09543.ap-south-1",
                user= snowflake_creds.USER_NAME,
                password= snowflake_creds.PASSWORD,
                role="ACCOUNTADMIN",
                warehouse="COMPUTE_WH",
                database="HEALTHDB",
                schema="HEALTHSCHEMA"
            ))
        
        mail_list = []
        
        with engine.connect() as conn:
    
            # Loading the scoring data
            score_data = pd.DataFrame(pd.read_sql(query,conn))
            score_data.columns = [col.upper() for col in score_data.columns.tolist()]
            mail_list.append('STEP-1: Loading Data complete')

            # Applying the preprocessing steps
            score_data_processed = LOS_Preprocessing.preprocess_data(score_data)
            mail_list.append('STEP-2: Applying the Preprocessing Steps complete')

            # Applying feature selection
            final_feats = pd.read_pickle('MODEL_FEATS.pkl')
            score_data_final = check_n_create_model_features(score_data_processed,final_feats)
            mail_list.append('STEP-3: Applying the Feature Selection Steps complete')

            # Getting the predictions
            model = xgboost.XGBRegressor()
            model.load_model('MODEL_XGB.model')
            score_data_final['PREDICTED_LOS'] = np.ceil(model.predict(score_data_final.drop('LOS',axis=1)))
            mail_list.append('STEP-4: Getting the Predictions complete')

            # Writing the dataframe to snowflake as a table
            score_data_final = score_data_final.reset_index()
            score_data_table = pd.merge(score_data,score_data_final,on='CASE_ID',how='left')
            status = insert_predictions_to_snowflake_table(score_data_table)
            mail_list.append('STEP-5: Writing the data to snowflake complete')
            
            # Creating the mail body
            mail_string = ",\n ".join(map(str, mail_list))
            send_status_mail(mail_string)
            print('Success')
    
    except:
        
        mail_content = 'Schedule is failed'
        send_status_mail(mail_content)
        print('Failed')
        

In [30]:
LOS_MODEL_DEPLOYMENT()

Success


In [ ]:
# Scheduling the notebook

import time
from datetime import datetime    
import pytz    
tz_NY = pytz.timezone('Asia/Kolkata')   
 

# scheduled hours in 24-hour format
hours_list = ["10:10","17:17"]
 
while True:
    tz_NY = pytz.timezone('Asia/Kolkata')   
    now = datetime.now(tz_NY)  # gets current datetime

    hour = str(now.hour) # gets current hour
    minute = str(now.minute) # gets current minute
    current_time = f"{hour}:{minute}" # combines current hour and minute

    # checks if current time is in the hours list
    if current_time in hours_list:
        LOS_MODEL_DEPLOYMENT()
    
    time.sleep(60) # waits a minute until it repeats


Success


KeyboardInterrupt: 

In [32]:
print(datetime.now())

2023-01-15 11:43:06.596398


In [33]:
tz_NY = pytz.timezone('Asia/Kolkata') 
print(datetime.now(tz_NY))

2023-01-15 17:13:37.830165+05:30


# Live Data Scoring Simulation:

In [24]:
# Creating the connection engine (way 1)
engine = create_engine(URL(
        account="tr09543.ap-south-1",
        user= snowflake_creds.USER_NAME,
        password= snowflake_creds.PASSWORD,
        role="ACCOUNTADMIN",
        warehouse="COMPUTE_WH",
        database="HEALTHDB",
        schema="HEALTHSCHEMA"
    ))

In [25]:
def scoring_query(a):
    query_sim = f"""

    WITH BASE AS (

        SELECT CASE_ID,
               COALESCE(HOSPITAL_CODE,0) AS HOSPITAL_CODE,
               COALESCE(HOSPITAL_TYPE_CODE,'None') AS HOSPITAL_TYPE_CODE,
               COALESCE(CITY_CODE_HOSPITAL,0) AS CITY_CODE_HOSPITAL,
               COALESCE(HOSPITAL_REGION_CODE,'None') AS HOSPITAL_REGION_CODE,
               COALESCE(AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL,0) AS AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL,
               COALESCE(DEPARTMENT,'None') AS DEPARTMENT,
               COALESCE(WARD_TYPE,'None') AS WARD_TYPE,
               COALESCE(WARD_FACILITY_CODE,'None') AS WARD_FACILITY_CODE,
               COALESCE(BED_GRADE,0) AS BED_GRADE,
               PATIENTID,
               COALESCE(CITY_CODE_PATIENT,0) AS CITY_CODE_PATIENT,
               COALESCE(TYPE_OF_ADMISSION,'None') AS TYPE_OF_ADMISSION,
               COALESCE(SEVERITY_OF_ILLNESS,'Minor') AS SEVERITY_OF_ILLNESS,
               COALESCE(VISITORS_WITH_PATIENT,0) AS VISITORS_WITH_PATIENT,
               COALESCE(AGE,'None') AS AGE,
               COALESCE(ADMISSION_DEPOSIT,0) AS ADMISSION_DEPOSIT,
               ADMISSION_DATE,
               DISCHARGE_DATE

        FROM HEALTHDB.HEALTHSCHEMA.SIMULATION_DATA

    ),

    BASE_WITH_FEATURES AS (

        SELECT *,
                MONTHNAME(ADMISSION_DATE) AS ADMISSION_MONTH,
                DAYNAME(ADMISSION_DATE) AS ADMISSION_DAY,
                CONCAT(TYPE_OF_ADMISSION,'-',SEVERITY_OF_ILLNESS) AS ADMISSION_ILLNESS,
                CONCAT(SEVERITY_OF_ILLNESS,'-',BED_GRADE) AS ILLNESS_BEDGRADE,
                CONCAT(DEPARTMENT,'-',SEVERITY_OF_ILLNESS) AS DEPARTMENT_ILLNESS,
                DATEDIFF(day,ADMISSION_DATE,DISCHARGE_DATE) AS LOS
        FROM BASE 

    )    

    SELECT * FROM BASE_WITH_FEATURES WHERE ADMISSION_DATE = CURRENT_DATE+{a}-46
    """
    return query_sim

In [26]:
def check_n_create_model_features(df,feat_list):
    test = pd.DataFrame()
    for col in feat_list:
        if col in df.columns.tolist():
            test[col] = df[col]
        else:
            test[col] = 0
    
    return test

In [27]:
def insert_predictions_to_snowflake_table(data):
    import pandas
    import snowflake.connector
    from snowflake.connector.pandas_tools import pd_writer, write_pandas

    engine = create_engine(URL(
            account="tr09543.ap-south-1",
            user= snowflake_creds.USER_NAME,
            password= snowflake_creds.PASSWORD,
            role="ACCOUNTADMIN",
            warehouse="COMPUTE_WH",
            database="HEALTHDB",
            schema="HEALTHSCHEMA"
        ))  
    
    # Creating the logging table if not exists already
    table = 'TEMP_LOS_PREDICTION_MODEL_LOGGING_TABLE_HARI'
    
    # Inserting the data to snowflake logging table
    data.to_sql(table, engine, index=False, if_exists='append', method=pd_writer)
    return 'Success'

In [28]:
def send_status_mail(mail_string):
    import mail_creds
    import smtplib
    from email.mime.multipart import MIMEMultipart
    from email.mime.text import MIMEText
    from email.mime.multipart import MIMEMultipart
    from email.mime.base import MIMEBase
    from email.mime.text import MIMEText
    from email.utils import formatdate
    from email import encoders
    
    subject = 'Patient LOS Prediction - STATUS MAIL'
    mail_content = mail_string

    username= mail_creds.MAIL_ID
    password= mail_creds.APP_PASSWORD
    send_from = mail_creds.MAIL_ID
    send_to = mail_creds.MAIL_ID
    Cc = ''
    msg = MIMEMultipart()
    msg['From'] = send_from
    msg['To'] = send_to
    msg['Cc'] = Cc
    msg['Date'] = formatdate(localtime = True)
    msg['Subject'] = subject
    msg.attach(MIMEText(mail_content, 'plain'))
    smtp = smtplib.SMTP('smtp.gmail.com',587)
    smtp.ehlo()
    smtp.starttls()
    smtp.login(username,password)
    smtp.sendmail(send_from, send_to.split(',') + msg['Cc'].split(','), msg.as_string())
    smtp.quit()    

In [29]:
def LOS_MODEL_DEPLOYMENT_SIM(x):
    try:
        import pickle
        import pandas as pd
        import numpy as np
        import sqlalchemy
        import snowflake.connector
        from sqlalchemy import create_engine

        import xgboost as xgb
        from datetime import datetime, timedelta
        import time  
        import pytz    
        tz_NY = pytz.timezone('Asia/Kolkata')

        import warnings
        warnings.filterwarnings('ignore')   
        
        import snowflake_creds
        import LOS_Preprocessing
        from LOS_Preprocessing import preprocess_data
        
        # Creating the connection engine (way 1)
        engine = create_engine(URL(
                account="tr09543.ap-south-1",
                user= snowflake_creds.USER_NAME,
                password= snowflake_creds.PASSWORD,
                role="ACCOUNTADMIN",
                warehouse="COMPUTE_WH",
                database="HEALTHDB",
                schema="HEALTHSCHEMA"
            ))
        
        mail_list = []
        
        with engine.connect() as conn:
    
            # Loading the scoring data
#             print(scoring_query(x))
            score_data = pd.DataFrame(pd.read_sql(scoring_query(x),conn))
            score_data.columns = [col.upper() for col in score_data.columns.tolist()]
            print(score_data['ADMISSION_DATE'].max())
            mail_list.append('STEP-1: Loading Data complete')

            # Applying the preprocessing steps
            score_data_processed = LOS_Preprocessing.preprocess_data(score_data)
            mail_list.append('STEP-2: Applying the Preprocessing Steps complete')

            # Applying feature selection
            final_feats = pd.read_pickle('MODEL_FEATS.pkl')
            score_data_final = check_n_create_model_features(score_data_processed,final_feats)
            mail_list.append('STEP-3: Applying the Feature Selection Steps complete')

            # Getting the predictions
            model = xgboost.XGBRegressor()
            model.load_model('MODEL_XGB.model')
            score_data_final['PREDICTED_LOS'] = np.ceil(model.predict(score_data_final.drop('LOS',axis=1)))
            mail_list.append('STEP-4: Getting the Predictions complete')

            # Writing the dataframe to snowflake as a table
            score_data_final = score_data_final.reset_index()
            score_data_table = pd.merge(score_data,score_data_final,on='CASE_ID',how='left')
            status = insert_predictions_to_snowflake_table(score_data_table)
            mail_list.append('STEP-5: Writing the data to snowflake complete')
            
            # Creating the mail body
            mail_string = ",\n ".join(map(str, mail_list))
            send_status_mail(mail_string)
            print('Success')
    
    except:
        
        mail_content = 'Schedule is failed'
        send_status_mail(mail_content)
        print('Failed')
        

In [30]:
LOS_MODEL_DEPLOYMENT_SIM(1)

2022-12-01
Success


In [31]:
# Scheduling the notebook (for live scoring simulation)

import time
from datetime import datetime    
import pytz    
tz_NY = pytz.timezone('Asia/Kolkata')   
 

# scheduled hours in 24-hour format
hours_list = ["10:10","17:17"]
 
for i in range(1,31):
#     tz_NY = pytz.timezone('Asia/Kolkata')   
#     now = datetime.now(tz_NY)  # gets current datetime

#     hour = str(now.hour) # gets current hour
#     minute = str(now.minute) # gets current minute
#     current_time = f"{hour}:{minute}" # combines current hour and minute

#     # checks if current time is in the hours list
#     if current_time in hours_list:
#         LOS_MODEL_DEPLOYMENT()
    
    # Run the scoring script
    LOS_MODEL_DEPLOYMENT_SIM(i)
    
    #time.sleep(60) # waits a minute until it repeats


2022-12-01
Success
2022-12-02
Success
2022-12-03
Success
2022-12-04
Success
2022-12-05
Success
2022-12-06
Success
2022-12-07
Success
2022-12-08
Success
2022-12-09
Success
2022-12-10
Success
2022-12-11
Success
2022-12-12
Success
2022-12-13
Success
2022-12-14
Success
2022-12-15
Success
2022-12-16
Success
2022-12-17
Success
2022-12-18
Success
2022-12-19
Success
2022-12-20
Success
2022-12-21
Success
2022-12-22
Success
2022-12-23
Success
2022-12-24
Success
2022-12-25
Success
2022-12-26
Success
2022-12-27
Success
2022-12-28
Success
2022-12-29
Success
2022-12-30
Success
